# PCS Riders Scrapper

Importamos librerías necesarias

In [2]:
# coding: utf8
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import os
import re
import sys
from IPython.display import clear_output

### Función principal de scrapping

In [3]:
user_agent = {'User-agent': 'Mozilla/5.0'}

# Funcion base para obtener el contendio HTML de una URL y parsearlo con BeautifulSoup
def download_soup(url):
    print('Empezando petición a: ' + url)
    response = requests.get(url, headers=user_agent)
    if response.status_code == 200:
        page = response.text
        soup = BeautifulSoup(page, 'lxml')
        print('Petición completada con éxito!')
        return soup
    else:
        print('Error, código de estado: '+ response.status_code)

### Obtención de URLs Equipos

In [3]:
def get_team_urls(year):
    # Creamos el directorio si no existe
    newdir = 'data/team_urls' 
    if not os.path.exists(newdir):
        os.makedirs(newdir)
    
    print("Obteniendo URL de equipos, año {}".format(year))
    url_str = "https://www.procyclingstats.com/teams.php?s=worldtour&year=" + str(year)
    # Obtenemos la lista de equipos
    startlist = download_soup(url_str).find_all('a', href = re.compile('team/'))
    # Obtenemos las URLs de los equipos
    urls = [x['href'] for x in startlist]
    list_team_urls = ["https://www.procyclingstats.com/" + u for u in urls if u[-4:] == str(year) ]
    
    # Guardamos las URLs en un archivo CSV
    df_teams = pd.DataFrame()
    df_teams["team_url"] = list_team_urls
    df_teams = df_teams.drop_duplicates("team_url")
    ouputfile_str = newdir + "/team_urls_" + str(year) + ".csv"
    df_teams.to_csv(ouputfile_str)
        
    print("URLs conseguidas, año {}".format(year))

In [4]:
# Obtenemos las URLs de los equipos del año indicado (en este caso 2024)
get_team_urls(2024)

Obteniendo URL de equipos, año 2024
Empezando petición a: https://www.procyclingstats.com/teams.php?s=worldtour&year=2024
Petición completada con éxito!
URLs conseguidas, año 2024


### Obtención de URL de corredores

In [ ]:
# Función para obtener las URLs de los ciclistas de cada equipo
# Utiliza el archivo creado con get_team_urls
# Descarga cada página de equipo y busca las URLs de los ciclistas
# Crea su propio archivo de URLs de ciclistas

def get_riders_urls(start_year, end_year):
    print("Obteniendo URLs de ciclistas")
    df_rider_urls = pd.DataFrame(columns=["rider_url"])

    # Iteramos sobre los años de los que tenemos URLs de equipos
    for year in range(start_year, end_year+1):
        df_team_urls = pd.read_csv("data/team_urls/team_urls_" + str(year) + ".csv")
        list_rider_urls = []

        # Iteramos sobre los equipos
        for idx, row in df_team_urls.iterrows():
            url = row["team_url"]
            # Corregimos un error en el nombre de un equipo
            if url=="https://www.procyclingstats.com/team/intermarche-circus-want-2024":
                print('Equipo no existente')
            else:
                # Descargamos la página del equipo
                timer = 0.5 + 0.5 * random.random()
                time.sleep(timer)
                riders = download_soup(url).find_all('td')
                riders = riders[:(29*3)]
                ciclistas = []
                for i in range(1, len(riders), 3):
                    ciclistas.append(riders[i].find_all('a'))

                # Expresión regular para encontrar el valor de href
                patron = r'href="(.*?)"'
                # Lista para almacenar los valores de href
                urls = []
                # Iterar sobre cada cadena en la lista
                for cadena in ciclistas:
                    cadena = str(cadena)
                    # Buscar el valor de href en la cadena
                    resultado = re.search(patron, cadena)
                    # Verificar si se encontró el valor y agregarlo a la lista
                    if resultado:
                        valor_href = resultado.group(1)
                        urls.append(valor_href)

                # Añadimos las URLs de los ciclistas a la lista
                for u in urls:
                    list_rider_urls.append('https://www.procyclingstats.com/' + u)
        # Creamos un DataFrame con las URLs de los ciclist
        tmp_rider_urls = pd.DataFrame()
        tmp_rider_urls["rider_url"] = list_rider_urls
        df_rider_urls = pd.concat([df_rider_urls, tmp_rider_urls])
        df_rider_urls = df_rider_urls.drop_duplicates("rider_url")
    # Guardamos el DataFrame en un archivo CSV
    df_rider_urls.to_csv("data/rider_urls.csv")
    print('URLs de ciclistas obtenidas')

In [ ]:
# Obtenemos las URLs de los ciclistas de los equipos de 2024
get_riders_urls(2024,2024)

Obteniendo URLs de ciclistas
Empezando petición a: https://www.procyclingstats.com/team/alpecin-deceuninck-2024
Petición completada con éxito!
Empezando petición a: https://www.procyclingstats.com/team/arkea-b-b-hotels-2024
Petición completada con éxito!
Empezando petición a: https://www.procyclingstats.com/team/ineos-grenadiers-2024
Petición completada con éxito!
Equipo no existente
Empezando petición a: https://www.procyclingstats.com/team/astana-qazaqstan-team-2024
Petición completada con éxito!
Empezando petición a: https://www.procyclingstats.com/team/bahrain-victorious-2024
Petición completada con éxito!
Empezando petición a: https://www.procyclingstats.com/team/bora-hansgrohe-2024
Petición completada con éxito!
Empezando petición a: https://www.procyclingstats.com/team/cofidis-2024
Petición completada con éxito!
Empezando petición a: https://www.procyclingstats.com/team/decathlon-ag2r-la-mondiale-2024
Petición completada con éxito!
Empezando petición a: https://www.procyclingsta

### Obtenemos datos de los ciclistas

In [6]:
# Función para obtener la información de un ciclista
# Utiliza el archivo creado con get_riders_urls
# Descarga cada página de ciclista y extrae la información
# Crea su propio archivo de información de ciclistas

def get_uniquerider_infos(rider_url):
    '''Función para obtener la información de un ciclista'''
    rider_soup = download_soup(rider_url)
    rider_infos = pd.Series()
    
### Nombre completo
    rider_infos["fullname"] = rider_soup.find('title').text
    
### Equipo
    try:
        team = rider_soup.find('span', class_='red').text
    except:
        team = 'noteam'
    rider_infos["team"] = team   
        
    inter_soup = rider_soup.find('div', {"class":"rdr-info-cont"})
    
### Fecha nacimiento
    list_birthdate = inter_soup.contents[1:4]
    rider_infos["birthdate"] = list_birthdate[0] + list_birthdate[2][:-5]

### País    
    try:
        country = inter_soup.find('a', class_='black').text.encode('latin-1', 'ignore').decode('utf-8', 'ignore')
    except(UnicodeEncodeError):
        country = inter_soup.find('a', class_='black').text
    rider_infos["country"] = country

### Altura    
    try:
        height = float(inter_soup.find(string='Height:').next.split()[0])
    except:
        height = 0
    rider_infos["height"] = height    

### Peso        
    try:
        weight = float(inter_soup.find(string='Weight:').next.split()[0])
    except:
        weight = 0
    rider_infos["weight"] = weight

### Puntuaciones por especialidad
    try:
        specialties = inter_soup.find('div', class_='pps').find_all('li')
        for i in range(len(specialties)):
            specialty = specialties[i].find('a').text
            points = int(specialties[i].find('div', class_='pnt').text)
            rider_infos[specialty] = points
    except:
        pass

### Puntos ranking UCI
    try:
        uci_rank = inter_soup.find(string='UCI World').next.next.text
    except:
        uci_rank = 0
    rider_infos["UCI_rank"] = uci_rank

### URL
    rider_infos["rider_url"] = rider_url

### Devolvemos la información del ciclista
    print('Completado:', rider_infos["fullname"])
    return pd.DataFrame({"fullname": rider_infos["fullname"], "team": rider_infos["team"], 
                         "birthdate": rider_infos["birthdate"], "country": rider_infos["country"], 
                         "height": rider_infos["height"], "weight": rider_infos["weight"], 
                         "Onedayraces": rider_infos["Onedayraces"], "GC": rider_infos["GC"], 
                         "TT": rider_infos["TT"], "Sprint": rider_infos["Sprint"], 
                         "Climber": rider_infos["Climber"], "Hills": rider_infos["Hills"], 
                         "UCI-Rank": rider_infos["UCI_rank"], "rider_url": rider_infos["rider_url"]}, index=[0])

In [9]:
# Probamos a extraer datos de un ciclista
get_uniquerider_infos("https://www.procyclingstats.com/rider/kaden-groves")

Empezando petición a: https://www.procyclingstats.com/rider/kaden-groves
Petición completada con éxito!
Completado: Kaden  Groves 


,fullname,team,birthdate,country,height,weight,Onedayraces,GC,TT,Sprint,Climber,Hills,UCI-Rank,rider_url
0,Kaden Groves,Alpecin - Deceuninck,23 December 1998,Australia,1.76,76.0,421,246,357,1321,150,1664,36,https://www.procyclingstats.com/rider/kaden-gr...


In [10]:
# Función para obtener la información de todos los ciclistas
# Utiliza el archivo creado con get_riders_urls
# Descarga cada página de ciclista y extrae la información
# Crea su propio archivo de información de ciclistas

def get_allriders_infos():
    '''Función para obtener la información de todos los ciclistas'''
    df_rider_urls = pd.read_csv("data/rider_urls.csv")
    df_rider_infos = pd.DataFrame(columns=["fullname", "team", "birthdate", "country", "height", "weight", "Onedayraces", "GC", "Sprint", "Climber", "TT", "Hills", "UCI-Rank", "rider_url"])
    for idx, row in df_rider_urls.iterrows():
        rider_url = row["rider_url"]
        rider_infos = get_uniquerider_infos(rider_url)
        df_rider_infos = pd.concat([df_rider_infos, rider_infos], ignore_index=True)
        df_rider_infos.to_csv("data/rider_infos.csv")
    print('Información de ciclistas obtenida')

# Obtenemos la información de todos los ciclistas
get_allriders_infos()

Empezando petición a: https://www.procyclingstats.com/rider/mathieu-van-der-poel
Petición completada con éxito!
Completado: Mathieu van der Poel 
Empezando petición a: https://www.procyclingstats.com/rider/jasper-philipsen


C:\Users\andre\AppData\Local\Temp\ipykernel_19192\3396933871.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_rider_infos = pd.concat([df_rider_infos, rider_infos], ignore_index=True)


Petición completada con éxito!
Completado: Jasper  Philipsen 
Empezando petición a: https://www.procyclingstats.com/rider/soren-kragh-andersen
Petición completada con éxito!
Completado: Søren Kragh Andersen 
Empezando petición a: https://www.procyclingstats.com/rider/silvan-dillier
Petición completada con éxito!
Completado: Silvan  Dillier 
Empezando petición a: https://www.procyclingstats.com/rider/kaden-groves
Petición completada con éxito!
Completado: Kaden  Groves 
Empezando petición a: https://www.procyclingstats.com/rider/xandro-meurisse
Petición completada con éxito!
Completado: Xandro  Meurisse 
Empezando petición a: https://www.procyclingstats.com/rider/gianni-vermeersch
Petición completada con éxito!
Completado: Gianni  Vermeersch 
Empezando petición a: https://www.procyclingstats.com/rider/quinten-hermans
Petición completada con éxito!
Completado: Quinten  Hermans 
Empezando petición a: https://www.procyclingstats.com/rider/ramon-sinkeldam
Petición completada con éxito!
Comp

### Obtenemos .csv del ranking UCI

In [7]:
def get_ranking(year):
    '''Función para obtener el ranking de ciclistas de un año'''
    print("Obteniendo ranking de ciclistas actual")
    if year == 2023:
        date = "2023-12-26"
    elif year == 2022:
        date = "2022-12-27"
    else:
        date = str(year)+"-12-31"
    

    # Obtenemos el ranking de los 100 primeros
    url_str = "https://www.procyclingstats.com/rankings.php?date="+date+"&nation=&age=&zage=&page=smallerorequal&team=&offset=0&teamlevel=&filter=Filter&p=me&s=uci-individual"
    ranking_soup = download_soup(url_str)
    ranking = ranking_soup.find_all('tr')
    ranking_list = []
    for i in range(1, len(ranking)):
        rider = ranking[i].find_all('td')
        rider_link = rider[3].find('a')
        # Hacemos scrapping del link del ciclista para obtener el nombre
        rider_url = "https://www.procyclingstats.com/" + rider_link['href']
        rider_name = get_uniquerider_infos(rider_url)["fullname"][0]
        rider_info = [rider_name, rider[4].text, rider[5].text]
        ranking_list.append(rider_info)
    ## Creamos un DataFrame con la información
    df_ranking = pd.DataFrame(ranking_list, columns=["rider", "team", "points"])

    # Obtenemos los siguientes 100 y almacenamos en el DataFrame
    url_str = "https://www.procyclingstats.com/rankings.php?date="+date+"&nation=&age=&zage=&page=smallerorequal&team=&offset=100&teamlevel=&filter=Filter&p=me&s=uci-individual"
    ranking_soup = download_soup(url_str)
    ranking = ranking_soup.find_all('tr')
    ranking_list = []
    for i in range(1, len(ranking)):
        rider = ranking[i].find_all('td')
        rider_link = rider[3].find('a')
        # Hacemos scrapping del link del ciclista para obtener el nombre
        rider_url = "https://www.procyclingstats.com/" + rider_link['href']
        rider_name = get_uniquerider_infos(rider_url)["fullname"][0]
        rider_info = [rider_name, rider[4].text, rider[5].text]
        ranking_list.append(rider_info)
    ## Creamos un DataFrame con la información
    df_ranking2 = pd.DataFrame(ranking_list, columns=["rider", "team", "points"])



    # Obtenemos el ranking de los 100 siguientes y almacenamos en el DataFrame
    url_str = "https://www.procyclingstats.com/rankings.php?date="+date+"&nation=&age=&zage=&page=smallerorequal&team=&offset=200&teamlevel=&filter=Filter&p=me&s=uci-individual"
    ranking_soup = download_soup(url_str)
    ranking = ranking_soup.find_all('tr')
    ranking_list = []
    for i in range(1, len(ranking)):
        rider = ranking[i].find_all('td')
        rider_link = rider[3].find('a')
        # Hacemos scrapping del link del ciclista para obtener el nombre
        rider_url = "https://www.procyclingstats.com/" + rider_link['href']
        rider_name = get_uniquerider_infos(rider_url)["fullname"][0]
        rider_info = [rider_name, rider[4].text, rider[5].text]
        ranking_list.append(rider_info)
    ## Creamos un DataFrame con la información
    df_ranking3 = pd.DataFrame(ranking_list, columns=["rider", "team", "points"])

    # Obtenemos los 100 siguientes y almacenamos en el DataFrame
    url_str = "https://www.procyclingstats.com/rankings.php?date="+date+"&nation=&age=&zage=&page=smallerorequal&team=&offset=300&teamlevel=&filter=Filter&p=me&s=uci-individual"
    ranking_soup = download_soup(url_str)
    ranking = ranking_soup.find_all('tr')
    ranking_list = []
    for i in range(1, len(ranking)):
        rider = ranking[i].find_all('td')
        rider_link = rider[3].find('a')
        # Hacemos scrapping del link del ciclista para obtener el nombre
        rider_url = "https://www.procyclingstats.com/" + rider_link['href']
        rider_name = get_uniquerider_infos(rider_url)["fullname"][0]
        rider_info = [rider_name, rider[4].text, rider[5].text]
        ranking_list.append(rider_info)
    ## Creamos un DataFrame con la información
    df_ranking4 = pd.DataFrame(ranking_list, columns=["rider", "team", "points"])

    # Obtenemos el ranking de los 100 siguientes y almacenamos en el DataFrame
    url_str = "https://www.procyclingstats.com/rankings.php?date="+date+"&nation=&age=&zage=&page=smallerorequal&team=&offset=400&teamlevel=&filter=Filter&p=me&s=uci-individual"
    ranking_soup = download_soup(url_str)
    ranking = ranking_soup.find_all('tr')
    ranking_list = []
    for i in range(1, len(ranking)):
        rider = ranking[i].find_all('td')
        rider_link = rider[3].find('a')
        # Hacemos scrapping del link del ciclista para obtener el nombre
        rider_url = "https://www.procyclingstats.com/" + rider_link['href']
        rider_name = get_uniquerider_infos(rider_url)["fullname"][0]
        rider_info = [rider_name, rider[4].text, rider[5].text]
        ranking_list.append(rider_info)
    ## Creamos un DataFrame con la información
    df_ranking5 = pd.DataFrame(ranking_list, columns=["rider", "team", "points"])

    # Concatenamos los DataFrames
    df_ranking = pd.concat([df_ranking, df_ranking2, df_ranking3, df_ranking4, df_ranking5], ignore_index=True)
    df_ranking.to_csv("data/ranking_" + str(year) + ".csv")

In [8]:
# Obtenemos el ranking UCI de 2023
get_ranking(2024)

Obteniendo ranking de ciclistas actual
Empezando petición a: https://www.procyclingstats.com/rankings.php?date=2024-12-31&nation=&age=&zage=&page=smallerorequal&team=&offset=0&teamlevel=&filter=Filter&p=me&s=uci-individual
Petición completada con éxito!
Empezando petición a: https://www.procyclingstats.com/rider/tadej-pogacar
Petición completada con éxito!
Completado: Tadej  Pogačar 
Empezando petición a: https://www.procyclingstats.com/rider/remco-evenepoel
Petición completada con éxito!
Completado: Remco  Evenepoel 
Empezando petición a: https://www.procyclingstats.com/rider/jasper-philipsen
Petición completada con éxito!
Completado: Jasper  Philipsen 
Empezando petición a: https://www.procyclingstats.com/rider/ben-o-connor
Petición completada con éxito!
Completado: Ben  O'Connor 
Empezando petición a: https://www.procyclingstats.com/rider/mathieu-van-der-poel
Petición completada con éxito!
Completado: Mathieu van der Poel 
Empezando petición a: https://www.procyclingstats.com/rider/